In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from datetime import datetime
from sklearn.preprocessing import power_transform

En este script, procedere a limpiar y procesar los datos. Estas tranformaciones forman parte del modelado oficial del proyecto.

Cada observacion dada en estas transformaciones se han visto analizadas previamente en los archivos del EDA.

- Estamos ante la primera version del modelo de recomendacion.

In [2]:
# Carga de datos
df_ratings = pd.read_parquet('../raw/data_model/ratings_train.parquet')
df_tags = pd.read_parquet('../raw/data_model/tags_train.parquet')

In [3]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9367814 entries, 0 to 9367813
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
 3   tag        object 
dtypes: float64(1), int64(2), object(1)
memory usage: 285.9+ MB


In [4]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000175 entries, 0 to 16000174
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     float64
 1   movieId    int64  
 2   rating     float64
 3   timestamp  float64
 4   title      object 
 5   genres     object 
dtypes: float64(3), int64(1), object(2)
memory usage: 732.4+ MB


Transformaciones de variables e ingenieria de caracteristicas en df_tags (lo siguiente se realiza tambien en testV1.0.parquet)

In [5]:
# Nos quedamos con el top tag mas relevante por cada movieId
df_tags = df_tags.groupby('movieId').apply(lambda x: x.nlargest(1, 'relevance')).reset_index(drop=True)

In [6]:
# Tranformamos distribucion relevance a normal, se encuentra sesgada hacia la derecha
def distribution_relevance_dfTags(df_tags):
    df_tags_copy = df_tags.copy()
    df_tags_copy['relevance']=power_transform(df_tags_copy[['relevance']], method='box-cox')
    return df_tags_copy

In [7]:
df_tags=distribution_relevance_dfTags(df_tags) # Realizamos transformacion de distribucion relevance

Fusionamos datasets df_tags y df_rating, poesteriormente guardamos en nuevo archivo

In [ ]:
df_fullData = df_ratings.merge(df_tags, how='right', on='movieId')

In [10]:
df_fullData.to_parquet('../raw/data_model/dataset_process_trainV1.0.parquet')# Guardamos dataset fusionado

Comienzo de procesamiento de datos en el conjunto de entrenamiento

In [2]:
df_ = pd.read_parquet('../raw/data_model/dataset_process_trainV1.0.parquet')#Carga de datos fusionados

In [3]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15839932 entries, 0 to 15839931
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     float64
 1   movieId    int64  
 2   rating     float64
 3   timestamp  float64
 4   title      object 
 5   genres     object 
 6   tagId      int64  
 7   relevance  float64
 8   tag        object 
dtypes: float64(4), int64(2), object(3)
memory usage: 1.1+ GB


In [4]:
# df_, revisando nulos
df_.isna().sum()

userId       12
movieId       0
rating       12
timestamp    12
title         2
genres        2
tagId         0
relevance     0
tag           0
dtype: int64

In [3]:
# Los valores nulos de df_ los modificaremos con el valor 0 y 'unknown'
def imputer_(df_):
    df_ratings_copy = df_.copy()
    
    # Columnas numéricas
    columns_numeric = ['userId', 'rating', 'timestamp']
    imp_numeric = SimpleImputer(strategy="constant", fill_value=0)
    df_ratings_copy[columns_numeric] = imp_numeric.fit_transform(df_ratings_copy[columns_numeric])

    # Columnas categóricas
    columns_categorical = ['title', 'genres']
    
    # Limpieza de valores nulos en columnas categóricas
    df_ratings_copy[columns_categorical] = df_ratings_copy[columns_categorical].fillna('unknown')

    # Imputación de valores constantes en columnas categóricas
    imp_categorical = SimpleImputer(strategy="constant", fill_value='unknown')
    df_ratings_copy[columns_categorical] = imp_categorical.fit_transform(df_ratings_copy[columns_categorical])

    return df_ratings_copy


In [4]:
df_= imputer_(df_)# llamada a funcion de limpieza de nulos 

In [7]:
# df_, revisando nulos despues de la imputacion
df_.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
tagId        0
relevance    0
tag          0
dtype: int64

In [8]:
df_[df_['rating']==0] # Peliculas no vistas por ningun usuario

,userId,movieId,rating,timestamp,title,genres,tagId,relevance,tag
15264903,0.0,64997,0.0,0.0,unknown,unknown,43,0.468197,alien
15835210,0.0,115991,0.0,0.0,unknown,unknown,610,0.024171,loneliness
15837321,0.0,117438,0.0,0.0,When Marnie Was There (2014),Animation|Drama,640,-0.157685,melancholic
15839696,0.0,127108,0.0,0.0,Brooklyn (2015),Comedy|Drama|Romance,563,1.122008,irish
15839697,0.0,127114,0.0,0.0,The End of the Tour (2015),Drama,303,-0.260872,dialogue driven
15839700,0.0,127130,0.0,0.0,Mistress America (2015),Comedy,846,-0.470699,relationships
15839704,0.0,127140,0.0,0.0,Sleeping with Other People (2015),Comedy,865,0.099736,romantic comedy
15839705,0.0,127146,0.0,0.0,Kurt Cobain: Montage of Heck (2015),Documentary,687,-0.088399,musicians
15839710,0.0,127164,0.0,0.0,"What Happened, Miss Simone? (2015)",Documentary,578,-0.842222,jazz
15839711,0.0,127178,0.0,0.0,99 Homes (2014),Drama,922,0.458299,single father


Referente a los valores nulos, hemos modificado los NaN a 0, dado que hay peliculas en el df_ratings que no han sido valoradas por ningun usuario.
Asi tambien, he modificado los NaN de las categoricas a 'unknown'.

In [9]:
# df_ratings, revisando duplicados en titulos (no queremos el mismo titulo mas de 1 vez)
df_.nunique() # Comprobamos los valores duplicados por columnas

userId         138494
movieId         10381
rating             11
timestamp    12537612
title           10380
genres            931
tagId             853
relevance        1018
tag               853
dtype: int64

El dataset se encuentran correctamente con respecto a los duplicados.
- En title no tenemos ningun titulo igual repetido. Lo unico repetido son los valores unknown
- En tags tenemos la misma cantidad de tag como de su id.

In [5]:
# Transformamos las variables del dataset 
# Timestamp, convertimos a datetime (referente a fecha de valoracion)
def timestamp_to_datetime_df_(df_):
    df_ratings_copy = df_.copy()
    # Convertimos el timestamp a datetime, luego aplicamos el formato año/mes/dia devolviendo un string
    df_ratings_copy['date_rating'] = df_ratings_copy['timestamp'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
    # Transformamos a Date el String
    df_ratings_copy['date_rating'] = df_ratings_copy['date_rating'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    # Eliminamos variable timestamp
    df_ratings_copy.drop('timestamp', axis=1, inplace=True)

    return df_ratings_copy

In [6]:
df_=timestamp_to_datetime_df_(df_) # Transforma timestamp a datetime (las fechas no valoradas se veran asi:"1970-01-01")

In [7]:
# Transformamos las variables del dataset df_
# date_rating, separamos año,mes y dia en variables independientes
def date_df_indepenDates(df_):
    df_ratings_copy = df_.copy()
    # Extraemos en variables sepadas el año, mes y dia de date_rating
    df_ratings_copy['year_rate'] = df_ratings_copy['date_rating'].dt.year
    df_ratings_copy['month_rate'] = df_ratings_copy['date_rating'].dt.month
    df_ratings_copy['day_rate'] = df_ratings_copy['date_rating'].dt.day
    # Eliminamos variable timestamp
    df_ratings_copy.drop('date_rating', axis=1, inplace=True)
    return df_ratings_copy

In [8]:
df_=date_df_indepenDates(df_)# Extraemos fechas independientes

In [9]:
# Transformamos las variables del dataset df_
# title, Extraemos el año de publicacion de cada pelicula en una variable nueva
def launchYear_title_df_(df_):
    df_ratings_copy = df_.copy()
    # Extraemos con expresiones regulares los años del title, rellenamos los nulos con 0 y casteamos a int
    df_ratings_copy['launch_year'] = df_ratings_copy['title'].str.extract(r'\((\d{4})\)', expand=False).fillna('0').astype(int)
    # Eliminamos el año y cualquier contenido entre paréntesis en la columna 'title'
    df_ratings_copy['title'] = df_ratings_copy['title'].apply(lambda x: x.split(' (')[0])
    return df_ratings_copy

In [10]:
df_=launchYear_title_df_(df_) # Extrae año del titulo

In [11]:
# Transformamos las variables del dataset df_
# Convertimos userId a entero
def int_userId_df_(df_):
    df_ratings_copy=df_.copy()
    # Convertir 'userId' a int
    df_ratings_copy['userId'] = pd.to_numeric(df_ratings_copy['userId'], errors='coerce').astype('int64')
    return df_ratings_copy

In [12]:
df_=int_userId_df_(df_)# Convierte userId a entero

In [13]:
# Transformamos las variables del dataset df_
# Extraemos la longitud de cada title y lo añadimos como nueva variable
def lenTitle_df_(df_):
    df_ratings_copy=df_.copy()
    #Extraemos la longitud de cada uno de los titulos
    df_ratings_copy['len_title']=df_ratings_copy['title'].apply(lambda x: len(x))

    return df_ratings_copy

In [14]:
df_=lenTitle_df_(df_) # Extraemos longitud del titulo

In [15]:
# Transformamos las variables del dataset df_
# Extraemos  el genero principal de cada movieId (se considera principal al primero que aparece en genres antes del '|')
def mainGenre_df_(df_):
    df_ratings_copy=df_.copy()
    # Extraemos el primer genero de la columna genres y lo almacenamos en una nueva variable
    df_ratings_copy['main_genre']=df_ratings_copy['genres'].apply(lambda x: x.split('|')[0])
    return df_ratings_copy


In [16]:
df_=mainGenre_df_(df_) #Extraemos genero principal

In [17]:
def valoration_df(df_):
    df_ratings_copy=df_.copy()
    # Gestionamos el rating en funcion de sus valores y añadimos cuatro categorias: 'excellent','good','bad','unknown'
    df_ratings_copy['valoration']=df_ratings_copy['rating'].apply(lambda x: 'excellent' if x == 5 else('good' if x >= 3 else ('unknown' if x == 0 else 'bad')))
    return df_ratings_copy

In [18]:
df_=valoration_df(df_)# Categorias en funcion de rating

Dataset procesado version 1